In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
img_id = "CA214032V0446"
calc_types = ["sat"]
overwrite = False

output_fld = "/data/ATM/data_1/georef"

In [3]:
# load the image
import src.load.load_image as li
image = li.load_image(img_id)

In [4]:
import src.base.connect_to_database as ctd

sql_string = f"SELECT * FROM images WHERE image_id='{img_id}'"
conn = ctd.establish_connection()
data = ctd.execute_sql(sql_string, conn)

month = data.iloc[0]['date_month']


In [11]:
import src.display.display_shapes as ds

import src.load.load_shape_data as lsd

path_sat_shapefile = "/data/ATM/data_1/georef/footprints/sat_footprints.shp"

# load the shapefile with the images
georef_shape_data = lsd.load_shape_data(path_sat_shapefile)

# get the geometry
georef_shapes = georef_shape_data.geometry

# get the unique ids from the shapes
georef_ids = georef_shape_data['image_id'].tolist()

print(georef_ids)


['CA214232V0168', 'CA164432V0027', 'CA183332V0109', 'CA181732V0012', 'CA183332V0136', 'CA182132V0089', 'CA184632V0357', 'CA196232V0038', 'CA184832V0105', 'CA180732V0011', 'CA181632V0103', 'CA181632V0139', 'CA183332V0148', 'CA183332V0135', 'CA182132V0150', 'CA181732V0014', 'CA184632V0356', 'CA183332V0119', 'CA183332V0069', 'CA174532V0013', 'CA183332V0117', 'CA184632V0370', 'CA216032V0031', 'CA214232V0154', 'CA214132V0070', 'CA212432V0069', 'CA183332V0134', 'CA181632V0111', 'CA184632V0369', 'CA207332V0125', 'CA214232V0157', 'CA212432V0074', 'CA216032V0025', 'CA212432V0080', 'CA182232V0034', 'CA184332V0031', 'CA181732V0021', 'CA181332V0083', 'CA183332V0118', 'CA184332V0053', 'CA183332V0080', 'CA181832V0003', 'CA181832V0054', 'CA181832V0002', 'CA183332V0150', 'CA181832V0045', 'CA181232V0072', 'CA182132V0155', 'CA184632V0381', 'CA196632V0033', 'CA183332V0147', 'CA214232V0162', 'CA180132V0123', 'CA182132V0152', 'CA181832V0046', 'CA216632V0291', 'CA181632V0038', 'CA181332V0085', 'CA181332V008

In [12]:
# Filter out geo-referenced images and footprints that don't match the flight path
filtered_georef_ids = []
filtered_georef_shapes = []
for i, geo_id in enumerate(georef_ids):
    if geo_id[2:6] == img_id[2:6]:
        filtered_georef_ids.append(geo_id)
        filtered_georef_shapes.append(georef_shapes[i])

# Replace the original lists with the filtered lists
georef_ids = filtered_georef_ids
georef_shapes = filtered_georef_shapes

print(georef_ids)

['CA214032V0429', 'CA214032V0430', 'CA214032V0437', 'CA214032V0451']


In [16]:
import src.georef.snippets.verify_image_position as vip

for i in range(len(georef_ids)):
    
    #deep copy list
    georef_shapes_copy = georef_shapes.copy()
    
    #remove the current image from the list
    georef_shapes_copy.pop(i)
        
    valid = vip.verify_image_position(georef_shapes[i], georef_shapes_copy)    


Distance from footprint to line: 19491.584038878027
19491.58403887806 19491.584038878027 4374.946680251785
PLTSHOW
Distance from footprint to line: 17113.716975256757
17113.716975256742 17113.716975256757 6803.255830280485
PLTSHOW
Distance from footprint to line: 1991.640711228045
1991.6407112279774 1991.640711228045 7439.063667033284
PLTSHOW
Distance from footprint to line: 22406.268017332764
22406.268017332764 22406.268017332764 6626.88372370274
PLTSHOW


In [10]:
import src.georef.georef_calc as gc

GCalc = gc.GeorefCalc()

transform, residuals, tps, conf = GCalc.georeference(image, img_id, georef_ids, georef_shapes)

PLTSHOW


<h4>Verify the image geometry and position</h4>

In [9]:
import src.georef.snippets.verify_image_geometry as vig
import src.georef.snippets.verify_image_position as vip

# check if the geometry of the image is valid
valid_geometry, reason = vig.verify_image_geometry(image, transform)

# check if the image fits in the position of the other images
#valid_position, reason = vip.verify_image_position(image, transform, georef_shapes)

print(valid_geometry)


True


<h4>Save the data</h4>

In [43]:
import numpy as np
import pandas as pd

import src.georef.snippets.apply_transform as af
import src.georef.snippets.convert_image_to_footprint as citf

import src.export.export_geometry as eg

# merge tps and conf
tps_conf = np.concatenate([tps, conf.reshape(-1, 1)], axis=1)

# define path to shape file
path_shp_file = f"{output_fld}/calc.shp"

# apply the transform and save geo-referenced tiff
af.apply_transform(image, transform, f"{output_fld}/calc/{img_id}.tif")

# save transform and points
np.savetxt(f"{output_fld}/calc/{img_id}_transform.txt", transform, fmt='%.5f')
np.savetxt(f"{output_fld}/calc/{img_id}_points.txt", tps_conf, fmt=['%i', '%i', '%.2f', '%.2f', '%.3f'])

# create a footprint for this image
footprint = citf.convert_image_to_footprint(image, transform)

# define attributes
attributes = {
    'image_id': img_id,
    'month': month,
    'num_tps': tps.shape[0],
    'avg_conf': round(np.mean(conf), 3),
    'avg_resi': round(np.mean(residuals), 3),
}

attributes = pd.DataFrame.from_dict(attributes, orient='index').T

# save footprint to shp file
eg.export_geometry(footprint, path_shp_file,
                   attributes=attributes, key_field="image_id",
                   overwrite_file=False,
                   overwrite_entry=True, attach=True)

print(f"{img_id} successfully saved")


CA184632V0333 successfully saved
